In [1]:
import pynbody
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [7]:
galaxy_masses=pd.DataFrame()
BH_halos=pd.DataFrame()
snap_id=0

In [27]:
def loader(path):
    '''returns snapshot and halo'''
    snap=pynbody.load(path)
    snap.physical_units()
    h=snap.halos()
    return snap,h

def BH_finder(snap):
    '''returns subsnap of black holes'''
    return snap.star[pynbody.filt.BandPass('tform','-15 Gyr','0 Gyr')]

def analysis(halo, view):
    '''center snap on halo's center of mass (angmom gives errors if there aren't enough pixels)'''
    if view==1:
        pynbody.analysis.angmom.faceon(halo)
    elif view==2:
        pynbody.analysis.angmom.sideon(halo)
    else:
        pynbody.analysis.halo.center(halo)
        
def plot_BH_pos(h, BH, grpid, attribute, snapshot, cmap='Greys', w=100, save=False, view=0):
    '''plots position of black holes. Send halo array(h), black hole array(BH), grpid, attribute, and snapshot you want to plot 
    Optional: width/preferred scope(w) (default=100), colormap(default='Greys'), save(default=False)'''
    
    pynbody.plot.image(h[grpid].s,qty=attribute,cmap=cmap, width=w, log=False)
    plt.plot(np.array((BH[np.where(BH['amiga.grp']==grpid)]['pos'])).T[0],np.array((BH[np.where(BH['amiga.grp']==grpid)]['pos'])).T[1],'r+')
    plt.xlim(-w/2,w/2)
    plt.ylim(-w/2,w/2)
    plt.title('%s_h%s_%s_w%s'%(snapshot, grpid, attribute, w))
    
    if save==True:
        plt.savefig('plots/%s/h%s/h148_h%s_%s_w%s.png'%(snapshot, grpid, grpid, attribute, w))
    plt.show()
    
def add_mass_columns(hgrpid, grpid, snap_id):
    '''returns row with gas, stellar, and total masses for one halo tracked through time'''
    df=pd.DataFrame([[hgrpid.g['mass'].sum(), hgrpid.s['mass'].sum(), hgrpid['mass'].sum()]], index=[snap_id], columns=['H[%d] Gas'%(grpid),'H[%d] Star'%(grpid),'H[%d] Total'%(grpid)])
    df.units='Msol'
    return df

def mass_df(h, BHgrp, snap_id):
    df=pd.DataFrame()
    for i in BHgrp:
        df=pd.concat([df, add_mass_columns(h[i], i, snap_id)], axis=1, join='outer')
    
    df.units='Msol'
    return df

def galaxy_df(snap_id_array,h,BHgrp):
    galaxy_masses=pd.DataFrame()
    for i in snap_id_array:
        vars()['mass_%s'%(i)]=mass_df(h,BHgrp,i)
        galaxy_masses=galaxy_masses.append(vars()['mass_%s'%(i)])

    return galaxy_masses

def plot_SFH(h, grpid,  snapshot, ls='dashed', c='k', save=True, mf=True):
    '''plots SFH. Send halo array(h[grpid]), linestyle(ls=dashed), color(c=k) grpid, and snapshot you want to plot 
    Optional:save(default=True)'''
    pynbody.plot.sfh(h[grpid].s,linestyle=ls,color=c, massform=mf);
    plt.title('%s_h[%s]_SFH'%(snapshot,grpid), y=1.14);
    plt.savefig('plots/SFH/%s_h[%s]_SFH'%(snapshot, grpid));
    
    
def auto_plot_SFH(h, BHgrp,  snapshot, lines='dashed', col='k', sve=True, mform=True):
    '''plots SFH. Send halo array(h), grpid, and snapshot you want to plot 
    Optional:linestyle(ls=dashed), color(c=k), save(default=True)'''
    for i in BHgrp:
        plot_SFH(h,i,snapshot, ls=lines,c=col, mf=mform, save=sve)

In [9]:
#array of last four digits for each snap
snap_id_array=['0139','0225','0275','0640','0974','1024','1269','1280','1408','1740','2048','2088','2432','2688','2816','2944','3072','3195','3200','3328','3456','3584','3606','3712','3840','3968','4096']

In [11]:
#snap id index (negative values start from end)
snap_id=snap_id-1
snap_id_array[snap_id]

'3968'

In [12]:
#set path
path='/data/scratch/jillian/h148/h148.cosmo50PLK.3072g3HbwK1BH.00%s/h148.cosmo50PLK.3072g3HbwK1BH.00%s'%(snap_id_array[snap_id],snap_id_array[snap_id])

In [13]:
#returns snapshot and halos in physical units (takes a couple of minutes)
snap,h=loader(path)

In [14]:
#returns subsnap of black holes
BH=BH_finder(snap)

In [ ]:
for i in np.unique(BH['amiga.grp']):
        plot_SFH(h,i,snap_id_array[snap_id], mf=False)

df=pd.DataFrame({'%s halo groups'%(snap_id_array[snap_id]):BH['amiga.grp'],'%s iords'%(snap_id_array[snap_id]):BH['iord']})
BH_halos=pd.concat([BH_halos, df], axis=1, join='outer')

BH_halos

vars()['mass_%s'%(snap_id_array[snap_id])]=mass_df(h,np.unique(BH['amiga.grp']),snap_id_array[snap_id])

galaxy_masses

galaxy_masses=galaxy_masses.append(vars()['mass_%s'%(snap_id_array[snap_id])])

galaxy_masses

#Use when plotting to center and adjust angle of perspective: view=1 for face on and 2 for side on, anything else will leave it as it is
view=0
analysis(h[grpid],view)

#plot location of black holes
plot_BH_pos(h, BH_test, grpid, attribute='tform', snapshot=snap_id_array[snap_id])

In [ ]:
for i in np.unique(BH['amiga.grp']):
    print i